In [14]:
import requests
import pandas as pd
from datetime import datetime
import json
import timeit
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# A window should pop up, use this to download the sentiment libraries for nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
companies = ['CPRX', 'BABA', 'SNE', 'APHA', 'GOOGL', 'MSFT', 'AMZN', 'NRZ', 'PTON', 'PYPL', 'NKE', 'V', 
             'UBER', 'GM', 'FB', 'KO', 'MRNA', 'NVDA', 'PLUG', 'TXMD', 'ZNGA', 'WMT', 'JNJ', 'NTDOY', 
             'DKNG', 'ET', 'PENN', 'GE', 'JPM', 'DIS']


start_dates = pd.date_range(start="2020-01-15",end="2020-08-14")
end_dates = pd.date_range(start="2020-01-16",end="2020-08-15")
start_dates = [date.strftime('%Y-%m-%d') for date in list(start_dates)]
end_dates = [date.strftime('%Y-%m-%d') for date in list(end_dates)]

news_data = {}

In [3]:
# Loading data from the finnhub.io API into the directory ./data/
calls = 0
start = timeit.timeit()
for company in companies:
    print(company)
    for i in range(len(start_dates)):
        if i % 60 == 0:
            print(start_dates[i], company)
        calls += 1
        r = requests.get('https://finnhub.io/api/v1/company-news?symbol=' + company + '&from=' + start_dates[i] + '&to=' + end_dates[i] + '&token=btndr0n48v6tvtde6mbg')
        f = open('./news_data/' + company + start_dates[i] + 'to' + end_dates[i] + '.json',"w")
        f.write(json.dumps(r.json()))
        f.close()
        if calls >= 60:
            calls = 0
            end = timeit.timeit()
            time.sleep(60 - (start - end))
            start = timeit.timeit()

CPRX
2020-01-15 CPRX


KeyboardInterrupt: 

In [4]:
# Retrieving data and placing into a dict for out pandas dataframe
headline_data = {}
headline_data['datetime'] = []
headline_data['date'] = []
headline_data['headline'] = []
headline_data['stock'] = []
headline_data['source'] = []
headline_data['summary'] = []

for company in companies:
    print(company)
    for i in range(len(start_dates)):
        with open('./news_data/' + company + start_dates[i] + 'to' + end_dates[i] + '.json', 'r') as openfile:
            json_object = json.load(openfile) 
        for news in json_object:
            headline_data['datetime'].append(news['datetime'])
            headline_data['date'].append(start_dates[i])
            headline_data['headline'].append(news['headline'])
            headline_data['stock'].append(news['related'])
            headline_data['source'].append(news['source'])
            headline_data['summary'].append(news['summary'])

CPRX
BABA
SNE
APHA
GOOGL
MSFT
AMZN
NRZ
PTON
PYPL
NKE
V
UBER
GM
FB
KO
MRNA
NVDA
PLUG
TXMD
ZNGA
WMT
JNJ
NTDOY
DKNG
ET
PENN
GE
JPM
DIS


In [5]:
# Converting dictionary to Pandas Dataframe
headline_data_df = pd.DataFrame.from_dict(headline_data)

In [7]:
# Saving dataframe
headline_data_df.to_csv('headlines.csv', index=False)

In [17]:
vader = SentimentIntensityAnalyzer()

In [18]:
# Applying the sentiment analysis to 'headline' column of our Pandas dataframe and placing
# into a list called scores
scores = headline_data_df['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

In [19]:
scores_df

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.286,0.714,0.000,-0.7184
2,0.286,0.714,0.000,-0.7184
3,0.105,0.800,0.095,-0.0516
4,0.105,0.800,0.095,-0.0516
...,...,...,...,...
333841,0.000,1.000,0.000,0.0000
333842,0.170,0.692,0.138,-0.1280
333843,0.000,0.706,0.294,0.3612
333844,0.000,1.000,0.000,0.0000


In [20]:
# Merging our headline dataframe with our scores dataframe
headline_data_df = headline_data_df.join(scores_df, rsuffix='_right')

In [21]:
headline_data_df

,datetime,date,headline,stock,source,summary,neg,neu,pos,compound
0,1579046400,2020-01-15,Implied Volatility Surging for Catalyst (CPRX)...,CPRX,Zacks,,0.000,1.000,0.000,0.0000
1,1579805117,2020-01-22,These unfair tax policies are putting a burden...,CPRX,https://business.financialpost.com,Ted Rechtshaffen: Some political titan needs t...,0.286,0.714,0.000,-0.7184
2,1579805117,2020-01-23,These unfair tax policies are putting a burden...,CPRX,https://business.financialpost.com,Ted Rechtshaffen: Some political titan needs t...,0.286,0.714,0.000,-0.7184
3,1580505387,2020-01-30,"With four houses worth $2.5 million, this Onta...",CPRX,https://business.financialpost.com,Real estate investing has been good to Oscar a...,0.105,0.800,0.095,-0.0516
4,1580505387,2020-01-31,"With four houses worth $2.5 million, this Onta...",CPRX,https://business.financialpost.com,Real estate investing has been good to Oscar a...,0.105,0.800,0.095,-0.0516
...,...,...,...,...,...,...,...,...,...,...
333841,1597412760,2020-08-14,AMC Theaters to Reopen Next Week With $0.15 Mo...,DIS,Yahoo,AMC Entertainment (NYSE: AMC) will reopen more...,0.000,1.000,0.000,0.0000
333842,1597412700,2020-08-14,Dow falls nearly 100 points on losses for shar...,DIS,Yahoo,DOW UPDATE The Dow Jones Industrial Average is...,0.170,0.692,0.138,-0.1280
333843,1597406400,2020-08-14,Apple&#39;s Services Bundle Might Finally Be R...,DIS,Yahoo,Investors have been hearing rumblings about Ap...,0.000,0.706,0.294,0.3612
333844,1597405920,2020-08-14,Cord-Cutting Is Accelerating — and Many Tradit...,DIS,Yahoo,"In the first quarter, subscriber counts for th...",0.000,1.000,0.000,0.0000
